# O Problema
Sliding Puzzle - Bloco Deslizante

In [ ]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

# Resolver o quebra-cabeças usando Buscas

In [3]:
from copy import deepcopy
import numpy as np
from itertools import combinations

class Estado:
    def __init__(self, array):
        self.array = np.array(array)
    
    def __repr__(self):
        return str(self.array[0]) + '\n' + str(self.array[1]) + '\n' + str(self.array[2]) + '\n'
    
    def __eq__(self, other):
        return (self.array == other.array).all()


class FilaPrioridade:
    def __init__(self):
        self.list = []

    def put(self, item, priority):
        self.list.append((priority, item))
        self.list.sort(key=lambda x: x[0])

    def get(self):
        return self.list.pop(0)[1]
    
    def EspacoVazio(self):
        return len(self.list) == 0

class No:
    def __init__(self, estadoInicial):
        self.estado = estadoInicial
        self.pos = self.getPos()
        self.caminho = [estadoInicial]
    
    def AlcancaNovoEstado(self, novoEstado):
        novoNo = deepcopy(self)
        novoNo.estado = novoEstado
        novoNo.pos = novoNo.getPos()
        novoNo.caminho.append(novoEstado)
        
        return novoNo
    
    def getPos(self):
        for i in range(3):
            for j in range(3):
                if self.estado.array[i][j] == 0:
                    return i, j

def VerificaPosicao(no, estadoParada):
    return no.estado == estadoParada

In [4]:
posicaoCorreta = [[1,2,3], [4,5,6], [7,8,0]]    # 0 é o espaço vazio
posicaoCorreta = Estado(posicaoCorreta)
print(posicaoCorreta)

[1 2 3]
[4 5 6]
[7 8 0]



## Busca em Largura

In [6]:
def BuscaLargura(estadoInicial, estadoParada):
    no = No(estadoInicial)
    frg = [no]
    cnt = 0
    while True:
        cnt += 1
        
        if frg == []:
            return False
        no = frg.pop(0)
        #Se a posição do quebra cabeça for a posição desejada paramos o laço
        if VerificaPosicao(no, estadoParada):
            print("Solução encontrada em {} movimentos".format(cnt))
            return no.caminho

        x, y = no.pos
        #A lógica da implementação para validação de um movimento é baseada em um laço
        #duplo de repetição. A primeira variável i representa a linha e j representa as colunas para aquela
        #linha. A validação do movimento é feita através da identificação da posição do número 0.
        #Uma vez encontrado adiciona-se e subtrai-se 1 no valor da linha para testar se é possível
        #movimentar para cima e para baixo. O mesmo teste é realizado para a coluna, porém nesse
        #caso é testado a possibilidade de mover-se para direita e esquerda. O movimento é valido se o
        #valor adicionado for menor que 3 ou se o valor subtraído for no mínimo 0.
        for i, j in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            if i+x < 0 or i+x >= 3 or j+y < 0 or j+y >= 3:
                continue
            novoEstado = deepcopy(no.estado)
            novoEstado.array[x+i][y+j], novoEstado.array[x][y] = novoEstado.array[x][y], novoEstado.array[x+i][y+j] 
            novoNo = no.AlcancaNovoEstado(novoEstado)
            if novoNo.estado in no.caminho:         
                continue
            frg.append(novoNo)

In [7]:
BuscaLargura(Estado([[1,2,3],[4,5,6],[7,0,8]]), posicaoCorreta)

Solução encontrada em 4 movimentos


[[1 2 3]
 [4 5 6]
 [7 0 8], [1 2 3]
 [4 5 6]
 [7 8 0]]

## Busca em Profundidade

In [2]:
def BuscaProfundidade(estadoInicial, estadoParada):
    no = No(estadoInicial)
    frg = [no]
    cnt = 0
    while True:
        cnt += 1
        
        if frg == []:
            return False
        no = frg.pop(0)
        if VerificaPosicao(no, estadoParada):
            print("Solução encontrada em {} movimentos".format(cnt))
            return no.caminho
        x, y = no.pos
        #Mesma lógica
        for i, j in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            if i+x < 0 or i+x >= 3 or j+y < 0 or j+y >= 3:
                continue
            novoEstado = deepcopy(no.estado)
            novoEstado.array[x+i][y+j], novoEstado.array[x][y] = novoEstado.array[x][y], novoEstado.array[x+i][y+j] #troca
            novoNo = no.AlcancaNovoEstado(novoEstado)
            if novoNo.estado in no.caminho: 
                continue
            frg.insert(0, novoNo)

In [5]:
BuscaProfundidade(Estado([[1,2,3],[4,5,6],[7,0,8]]), posicaoCorreta)

Solução encontrada em 2 movimentos


[[1 2 3]
 [4 5 6]
 [7 0 8], [1 2 3]
 [4 5 6]
 [7 8 0]]

## Discorra sobre o desempenho dos métodos em questões de:


1.   Consumo de memória
2.   Processamento

Sabemos que para chegar à uma solução do problema do quebra-cabeça de blocos deslizantes partimos de um estado inicial qualquer, pode ser qualquer combinação de números entre 0-8, onde 0 representa vazio, até um estado final desejado. 
Em geral, no algoritmo criado a Busca em Profundidade visita todos os vértices do grafo, enquanto a Busca em Largura visita apenas os vértices que estão ao alcance do vértice inicial.

---
Em termos de consumo de memória a Busca em Profundidade tem uma vantagem considerável, pois apenas o caminho de nós sendo analisados precisa armazenado. Caminhos que já foram explorados podem ser descartados da memória. Contudo, em termos de desempenho, existe uma desvantagem grande em relação à busca em largura, pois o algoritmo de busca em profundidade pode fazer uma busca muito 
longa mesmo quando a resposta do problema está localizada a poucos nós da raiz da árvore.

Em relação aos cenários testados em ambos os métodos, conclui que, para definições do jogo de quebra cabeça onde as posições não estão muito diferentes da posição inicial, a busca por largura pode ser pode ser considerada um método eficaz, porém quando temos uma posição mais complexa não temos uma solução em perfomance de tempo satisfatória. A busca por profundidade, por outro lado, solucionou poucos cenários propostos.


